In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data=pd.read_csv('gdrive/My Drive/RandomForest/Iris.csv',index_col='Id')
data

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
Id,,,,,
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,Iris-virginica
147,6.3,2.5,5.0,1.9,Iris-virginica
148,6.5,3.0,5.2,2.0,Iris-virginica


In [5]:
data_types = data.dtypes
data_types = list(data_types)

In [6]:
# Separate features and target variable

X = data.iloc[:,:-1].to_numpy()
Y = np.array(data.iloc[:,-1])



In [7]:
def splitData(X,Y,testPercentage= 0.2):
      trainPercentage = 1 - testPercentage
      Xtrain = []
      Ytrain = []
      Xtest = []
      Ytest = []

      labels , counts  = np.unique(Y,return_counts=True)
      # print(labels,counts)

      for i in range(len(labels)):
        indices = np.where(Y == labels[i])
        Xt = X[indices]
        Yt = Y[indices]
        # print(Xt)
        # print(Yt)
        randomArray = np.arange(0,counts[i])
        np.random.shuffle(randomArray)
        # print(randomArray)
        # print(round(trainPercentage * counts[i]))
        Xtrain_indices_After_shuffling = randomArray[0:(round(trainPercentage * counts[i]))]
        Xtest_indices_After_shuffling = randomArray[(round(trainPercentage * counts[i])): counts[i]]
        Xtrain_shuffle = Xt[Xtrain_indices_After_shuffling]
        Xtest_shuffle =  Xt[Xtest_indices_After_shuffling]

        Xtrain.extend(Xtrain_shuffle)
        Ytrain.extend(Yt[Xtrain_indices_After_shuffling])
        Xtest.extend(Xtest_shuffle)
        Ytest.extend(Yt[Xtest_indices_After_shuffling])


      Xtrain = np.array(Xtrain)
      Ytrain = np.array(Ytrain)
      Xtest = np.array(Xtest)
      Ytest = np.array(Ytest)

      return Xtrain,Ytrain,Xtest,Ytest


In [8]:
# Function to split the dataset
Xtrain,Ytrain,Xtest,Ytest = splitData(X,Y)
print (" Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape)
print (" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytest.shape)


 Training Data Set Dimensions= (120, 4) Training True Class labels dimensions (120,)
 Test Data Set Dimensions= (30, 4) Test True Class labels dimensions (30,)


In [9]:
class Node:
    def __init__(self, split_point = None):
        self.split = split_point
        self.fidx = None
        self.score = None
        self.label = None
        self.purity = None
        self.left = None
        self.right = None


    # def __init__(self)


class Tree:
    def __init__(self):
        self.root = None
        # self.traceChild = None




    def set_childs(self, bsp,idx,bss,child,trace_node):
        print(child)

        if self.root is None:
            new_node = Node()
            self.root = new_node
            trace_node = new_node
            trace_node.split = bsp
            trace_node.score = bss
            trace_node.fidx = idx
            return trace_node

        if(child == "left"):
            print('L')
            trace_node.left = Node()
            trace_node = trace_node.left
            trace_node.split = bsp
            trace_node.score = bss
            trace_node.fidx = idx
            return trace_node

        else:
            print("R")
            trace_node.right = Node()
            trace_node = trace_node.right
            trace_node.split = bsp
            trace_node.score = bss
            trace_node.fidx = idx
            return trace_node

    def leaf_node(self,purity,child,trace_node,label):
        print(child)
        if(child == "left"):
            trace_node.left = Node()
            trace_node = trace_node.left
            trace_node.purity = purity
            trace_node.label = label
            return trace_node
        else:
            trace_node.right = Node()
            trace_node = trace_node.right
            trace_node.label = label
            return trace_node

    def predict(self,example):

        temp = self.root
        while True:
            if(temp.label is None):
                if(example[temp.index] < temp.data):
                    temp = temp.left
                else:
                    temp = temp.right
            else:
                return temp.label




    def level_order(self):
        queue = [self.root]
        if self.root is None:
            return
        while queue:
            node = queue.pop(0)
            if(node.label == None):
               print(node.data,",",node.index, end=" : ")
            else:
                print(node.label, end=" ")
            if node.left is not None:
                # print('left')
                queue.append(node.left)
            if node.right is not None:
                # print("right")
                queue.append(node.right)











In [10]:


class DecisionTree:
    def __init__(self,data_types,purity=0.90,extrathreshold=5,maxdepth=5):
        print("Construcor")
        self.tree = Tree()
        self.node = Node()
        self.data_types = data_types
        self.purity = purity
        self.extrathreshold = extrathreshold
        self.maxdepth = maxdepth
    def categorical_splitPoints(self,feature,Class):
        split_points= np.unique(feature)
        classes = np.unique(Class)
        print("Split Points : ",split_points)
        print("classes : ",classes)
        tempArray = np.zeros((len(split_points),len(classes)))
        print(tempArray.shape)
        for i in range(len(split_points)):
            print(i)
            idx = np.where(split_points[i] == feature)
            data_At_split_point = Class[idx]
            labels,counts = np.unique(data_At_split_point,return_counts=True)
            print(labels,counts)
            try:
                tempArray[i,:] = counts

            except ValueError:
                print("Except")
                m = 0
                for j in range(0,len(classes)):
                    if(classes[j] in labels):
                        tempArray[i][j] = counts[m]
                        m+=1
                    else:
                        tempArray[i][j] = 0


        # print(tempArray)
        return np.array(tempArray)

    def Categorical_Entropy(self,feature,Class):
        unique_values= np.unique(feature)
        split_points_matrix = self.categorical_splitPoints(feature,Class)
        labels,counts = np.unique(Class,return_counts=True)
        # print(unique_values)
        # print(labels,counts)
        # print(np.sum(split_points_matrix[0]))
        # print(np.sum(split_points_matrix[1]))
        # for i in range(split_points_matrix.shape[0]):
            # if(i>0):
        entropy = 0
        left_entropy = 0
        right_entropy = 0
        for i in range(0,(split_points_matrix).shape[1]):
            if(split_points_matrix[0][i]!=0):
                # print(split_points_matrix[0][i])
                left_entropy += (split_points_matrix[0][i]/np.sum(split_points_matrix[0]))*(np.log(split_points_matrix[0][i]/sum(split_points_matrix[0])))

            if(split_points_matrix[1][i]!=0):
                # print(split_points_matrix[1][i])
                right_entropy += (split_points_matrix[1][i]/np.sum(split_points_matrix[1]))*(np.log(split_points_matrix[1][i]/sum(split_points_matrix[1])))

        Lentropy = -1 * (np.sum(split_points_matrix[0])/np.sum(counts)) * left_entropy
        Rentropy  = -1 * (np.sum(split_points_matrix[1])/np.sum(counts)) * right_entropy
        entropy = Lentropy + Rentropy
        # print("Entropy : ",entropy)
        bsp = unique_values[0]
        bss = entropy
        left_data = np.isin(feature,bsp)
        right_data = np.isin(feature,bsp,invert=True)
        return bsp,bss,left_data,right_data


    def findSplitPoints(self,feature):
        # print("Split")
        # print(len(np.unique(feature)))
        unique_sorted = np.unique(feature)
        return (unique_sorted[:-1] + unique_sorted[1:]) / 2


    def Entropy(self,left_data,labels_counts):

        right_data = labels_counts - left_data
        left_entropy = 0
        right_entropy = 0
        for i in range(0,len(left_data)):
            if(left_data[i]!=0):
                left_entropy += (left_data[i]/sum(left_data))*(np.log(left_data[i]/sum(left_data)))

            if(right_data[i]!=0):
                right_entropy += (right_data[i]/sum(right_data))*(np.log(right_data[i]/sum(right_data)))

        Lentropy = -1 * (sum(left_data)/sum(labels_counts)) * left_entropy
        Rentropy  = -1 * (sum(right_data)/sum(labels_counts)) * right_entropy
        entropy = Lentropy + Rentropy
        return entropy
    def CalculateEntropy(self,feature,Class):

        split_point = self.findSplitPoints(feature)
        # split_point.append(0)
        print(len(split_point))

        sorted_feature = np.sort(feature)
        # print(sorted_feature)
        sorted_indices = np.argsort(feature)
        sorted_labels = np.array(Class)[sorted_indices]
        labels,counts = np.unique(sorted_labels,return_counts=True)
        # print(labels,counts)
        entropy = []
        temp_array = np.zeros(len(labels))
        # data_array = np.zeros((len(split_point)+1,len(labels)))
        k = 0
        # print(len(sorted_feature))

        for i in range(len(sorted_feature)-1):
            # print(i)
            # print(sorted_feature[i],"  ",sorted_feature[i+1])
            for j in range(len(labels)):
                if(sorted_labels[i]==labels[j]):
                    temp_array[j]+=1


            if(((sorted_feature[i] + sorted_feature[i+1])/2) == split_point[k]):
                # print('K',k)
                # print(split_point[k])
                # data_array[k,:] = temp_array
                # print("data Array")
                k+=1

                entropy.append(self.Entropy(np.array(temp_array),np.array(counts)))
                    # print("Run Entropy")
            if(k==len(split_point)):
                break

        if(k==0):
            entropy.append(self.Entropy(np.array(temp_array),np.array(counts)))


            # print("Afterloop")
            # data_array[k,:] = counts
        # print (min(entropy))
        # print (split_point[np.argmin(entropy)])
        bsp = split_point[np.argmin(entropy)]
        bss = min(entropy)
        left_data = np.where(feature<=bsp)
        right_data = np.where(feature>bsp)
        return bsp,bss,left_data,right_data




    # def train(self,Xtain,Ytrain):

    def buildTree(self,Xtrain,Ytrain,traceChild,child,depth):
        n = len(Xtrain)
        labels,counts = np.unique(Ytrain,return_counts=True)
        # print("Max : ",max(counts))

        if(len(counts>0)):
            purity = max(counts)/n

        if(n<self.extrathreshold or purity > self.purity or  depth > self.maxdepth):
            print("in if : ",child)
            if(n == 0):
                traceChild = self.tree.leaf_node(child,traceChild,2)

            else:
                traceChild = self.tree.leaf_node(purity,child,traceChild,labels[np.argmax(counts)])
            return


        bsp = np.inf
        bss = np.inf
        fbidx = np.inf
        for i in range(Xtrain.shape[1]):
            # print(Xtrain[:,i])
            # print(self.data_types[i])
            if(self.data_types[i] == 'object'):
                # print("Categorical")
                print(" I : ",i)
                fbsp,fbss,left_data,right_data = self.Categorical_Entropy(Xtrain[:,i],Ytrain)
            else:
                print(" Numerical")
                print(" I : ",i)
                fbsp,fbss,left_data,right_data = self.CalculateEntropy(Xtrain[:,i],Ytrain)

            if(fbss<bss):
                print("BSS : ",bss,"  fbss : ",fbss)
                print("BSP : ",bsp,"  fbsp : ",fbsp)
                print("Index : ",i)
                bss = fbss
                bsp = fbsp
                fbidx = i

        # print("parent split point : ",sp)
        # # print(fbidx)


        Xtrain_left = Xtrain[left_data]
        Ytrain_left = Ytrain[left_data]
        Xtrain_right = Xtrain[right_data]
        Ytrain_right = Ytrain[right_data]
        print("X Left : ",Xtrain_left.shape)
        print("Y Left : ",Ytrain_left.shape)
        print("X Right : ",Xtrain_right.shape)
        print("Y Right : ",Ytrain_right.shape)

        traceChild = self.tree.set_childs(bsp,fbidx,bss,child,traceChild)
        # print("Tree : ",tree.level_order())
        self.buildTree(Xtrain_left,Ytrain_left,traceChild,'left',depth+1)
        self.buildTree(Xtrain_right,Ytrain_right,traceChild,'right',depth+1)

    def train(self,X,Y):
      print("train")
      self.buildTree(X,Y,self.node,'left',0)


    def predict(self,Xtest):
        print("test")

        nexamples,nfeatures = Xtest.shape
        predictions_1 = []
        predictions_2 = []
        for i in range(nexamples):
            predictions_1.append(self._predict(Xtest[i]))
            # predictions_2.append(self.tree.predict(Xtest[i]))

        return predictions_1

    def _predict(self,example):
      temp = self.tree.root
      while True:
          if(temp.label is None):
              if(self.data_types[temp.fidx] != 'object'):
                  if(example[temp.fidx] < temp.split):
                      temp = temp.left
                  else:
                      temp = temp.right
              else:
                  if(example[temp.fidx] in temp.split):
                      temp = temp.left
                  else:
                      temp = temp.right

          else:
              return temp.label

    def PrintTree(self):
        # print("Tree root : ",self.tree.root)

        print()
        print("IN ")
        queue = [self.tree.root]
        if self.tree.root is None:
            return
        while queue:
            node = queue.pop(0)
            if(node.label == None):
               print('I(Fidx={},Score={},Split={})'.format(node.fidx,node.score,node.split))
            else:
                print('C(class={},Purity={})'.format(node.label,node.purity))
            if node.left is not None:
                # print('left')
                queue.append(node.left)
            if node.right is not None:
                # print("right")
                queue.append(node.right)












In [11]:

DT = DecisionTree(data_types)
DT.train(Xtrain,Ytrain)

Construcor
train
 Numerical
 I :  0
33
BSS :  inf   fbss :  0.7408094406669399
BSP :  inf   fbsp :  5.55
Index :  0
 Numerical
 I :  1
22
 Numerical
 I :  2
40
BSS :  0.7408094406669399   fbss :  0.46209812037329684
BSP :  5.55   fbsp :  2.45
Index :  2
 Numerical
 I :  3
21
X Left :  (40, 4)
Y Left :  (40,)
X Right :  (80, 4)
Y Right :  (80,)
left
in if :  left
left
 Numerical
 I :  0
26
BSS :  inf   fbss :  0.5606417296428975
BSP :  inf   fbsp :  6.15
Index :  0
 Numerical
 I :  1
14
 Numerical
 I :  2
31
BSS :  0.5606417296428975   fbss :  0.26304962972352575
BSP :  6.15   fbsp :  4.95
Index :  2
 Numerical
 I :  3
15
BSS :  0.26304962972352575   fbss :  0.19289617755759103
BSP :  4.95   fbsp :  1.75
Index :  3
X Left :  (42, 4)
Y Left :  (42,)
X Right :  (38, 4)
Y Right :  (38,)
right
R
in if :  left
left
in if :  right
right


In [12]:
DT.PrintTree()


IN 
I(Fidx=2,Score=0.46209812037329684,Split=2.45)
C(class=Iris-setosa,Purity=1.0)
I(Fidx=3,Score=0.19289617755759103,Split=1.75)
C(class=Iris-versicolor,Purity=0.9285714285714286)
C(class=Iris-virginica,Purity=None)


In [13]:
pred = DT.predict(Xtest)

Acc = np.sum(pred == Ytest)/Ytest.shape[0]
# Acc2 = np.sum(pred == Ytest)/Ytest.shape[0]
print(Acc)

test
0.9333333333333333
